In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import numpy as np
import transformers
from transformers import BertTokenizer
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
import re
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yt-comments-bias-tips-normal/learn_yt_comments.csv
/kaggle/input/dataset-for-testing-yt-comms-bias/test_yt_comments.csv
/kaggle/input/fine-tune-chat-gpt/learn_yt_comments fine-tune.csv


In [174]:
# THIS SCRIPT ANALYZES YOUTUBE COMMENTS

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', use_fast=False)
labels = {'extra': 0,
          'normal': 1,
          'question': 2,
          'timestamps': 3,
          'issue': 4
          }
labels2 = ['extra', 'normal', 'question', 'timestamps', 'issue']

df = pd.read_csv("../input/fine-tune-chat-gpt/learn_yt_comments fine-tune.csv")
df_test = pd.read_csv("../input/dataset-for-testing-yt-comms-bias/test_yt_comments.csv")

df.head()

i = 0
for text in df['text']:
    text = re.sub("[.,!?-]", '', text.lower())
    text = re.sub("\s{2,}", ' ', text.lower())
    df['text'][i] = text
    i+=1

np.random.seed(75)
df_train, df_val = np.split(df.sample(frac=1, random_state=12), [int(.95*len(df))])

print(df_train.head())
print(df_train.at[2, 'category'])

print(len(df_train), len(df_val), len(df_test))

       category                                               text
19440    normal  you should check out the book disclosure proto...
830       extra  i don't know if you did it intentionally or no...
16233  question  i still have problems with characters that hav...
4509     normal  even these have the typical 'midjourney' style...
570       extra  filtering the labels based on the confidence l...
extra
18701 985 1038


In [175]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    
    text = text.lower()
    text = re.sub(r"what's", " what is ", text)
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", " can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", " i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)
    
    text = re.sub(r'https?:\/\/[\w\-!@#$%\/\^&*\.\(\)\d=]+', ' extra_link ', text)  # replace links with extra_link
    text = re.sub(r'\w+(-{1,2}[\w]{2,}){1,}', ' extra_command ', text) # dont use apt-get command` => 'apt-get' goes => 'extra_command'
    text = re.sub(r'(-{1,2}[\w]{2,}){1,}', ' extra_argument ', text) # extra, the `--no-half` => '--no-half' goes => 'extra_argument'
    text = re.sub(r'\d{1,2}:(\d\d?:?){1,}', ' extra_timestamp ', text) # 10:01 to 10:40:3 => extra_timestamp to extra_timestamp
    text = re.sub(r'\d+',' extra_number ', text) # replace numbers with extra_number
    text = re.sub(r'=', ' equals ', text) # replace = with equals
    
    text = re.sub(r'[^\w\s]',' ', text)
    #text = re.sub(r'\s+[a-zA-Z]\s+',' ', text)
    text = re.sub(r' the ',' ', text)
    text = re.sub(r' and ',' ', text)
    text = re.sub(r' or ',' ', text)
    text = re.sub(r"-", " - ", text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [176]:
print(text_preprocessing("extra, knowing that alfa+beta=127 for the reasons mentioned i have considered the quadrilateral efcd from which x + 53 + 180 - alpha + 180 - beta = 360 that easily leads to x=74"))
print(text_preprocessing("dude it's c1 = a1b1 + a2b3 it's written literally under the picture..."))
print(text_preprocessing("extra,1:04:34 for training very deep transformers see https://arxiv.org/pdf/2003.04887.pdf"))
print(text_preprocessing("extra,2:57 you mean \alpha_n = \frac{1}{\lambda} (y_n - w^t\phi(x_n)) without the \sum operator. had me confused for a while."))
print(text_preprocessing("extra,@ 28:32 protoattend ( https://arxiv.org/abs/1902.06292) is what you're looking for"))
print(text_preprocessing("extra,commands: open powershell as admin set-executionpolicy remotesigned install-module pswindowsupdate import-module pswindowsupdate get-windowsupdate install-windowsupdate"))
print(text_preprocessing("extra,adding --seed --stylize @ 16:17  just wonder if using a --seed referenced to a previous render of boy-girl-rabbit-pool along with the two image references could help maintain character consistencies: hair clothing (style & color) as well as introducing a lower level of --stylize 50 to reduce the midjourney induced variability within the new renderings any ideas wish i had the time to play with midjourney. you do good work! thank you. ( --stylize per midjourney: low stylization values produce images that closely match the prompt but are less artistic. high stylization values create images that are very artistic but less connected to the prompt.)"))
print(df_train['text'][15:35])
for i in df_train.index:
    df_train['text'][i]=text_preprocessing(df_train['text'][i])
print(df_train['text'][15:35])

extra knowing that alfa beta equals extra_number for reasons mentioned i have considered quadrilateral efcd from which x extra_number extra_number alpha extra_number beta equals extra_number that easily leads to x equals extra_number
dude it is c extra_number equals a extra_number b extra_number a extra_number b extra_number it is written literally under picture
extra extra_timestamp for training very deep transformers see extra_link
extra extra_timestamp you mean lpha_n equals rac extra_number lambda y_n w t phi x_n without sum operator had me confused for a while
extra extra_timestamp protoattend extra_link is what you are looking for
extra commands open powershell as admin extra_command remotesigned extra_command pswindowsupdate extra_command pswindowsupdate extra_command extra_command
extra adding extra_argument extra_argument extra_timestamp just wonder if using a extra_argument referenced to a previous render of extra_command along with two image references could help maintain ch

In [177]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [178]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def __len__(self):
        return len(self.texts)

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)

        return batch_texts

In [179]:
class BertClassifier(nn.Module):
    
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        linear_output = self.linear(pooled_output)
        relu = self.relu(linear_output)
        return relu

In [180]:
def train(model, train_data, val_data, learning_rate, epochs):
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    criterion = nn.CrossEntropyLoss()
    
    if use_cuda:
        model = model.cuda()
        
    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0
        
        optimizer = Adam(model.parameters(), lr=learning_rate)

        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0
        

        with torch.no_grad():
            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
        torch.save(model.state_dict(), "../working/model_yt.pt")

In [ ]:
EPOCHS = 1
LR = 3e-5
gc.collect()
torch.cuda.empty_cache()
model = BertClassifier()
model.load_state_dict(torch.load("../working/model_yt.pt"))

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 585/585 [03:31<00:00,  2.77it/s]


Epochs: 1 | Train Loss:  0.002                 | Train Accuracy:  0.985                 | Val Loss:  0.005                 | Val Accuracy:  0.960


 29%|██▊       | 168/585 [01:00<02:30,  2.78it/s]

In [153]:
def evaluate(model, test_data):
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        i = 0
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            print(nn.Softmax(output))

            acc = (output.argmax(dim=1) == test_label).sum().item()
            if labels2[output.argmax(dim=1).item()] != labels2[test_label.item()]:
                print(labels2[output.argmax(dim=1).item()] + ',' + test_data.at[i, 'text'])
            else:
                print(test_data.at[i, 'category'] + ',' + test_data.at[i, 'text'])
                #print(tokenizer.decode(test_dataloader.dataset.texts[i].input_ids[0])[:200], "| IS | ", labels2[output.argmax(dim=1).item()], " : ", labels2[test_label.item()])
            total_acc_test += acc
            i += 1

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [155]:
def evaluateAndPrintForModel(model, test_data):
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        i = 0
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            #print(output[0])
            #print(output[0][0].item())
            #print(output[0][1].item())
            #print(output[0][2].item())
            #print(output[0][3].item())
            #print(output[0][4].item())
            #return
            if labels2[output.argmax(dim=1).item()] != labels2[test_label.item()]:
                print(labels2[output.argmax(dim=1).item()] + ',' + test_data.at[i, 'text'])
            else:
                print(test_data.at[i, 'category'] + ',' + test_data.at[i, 'text'])
                #print(tokenizer.decode(test_dataloader.dataset.texts[i].input_ids[0])[:200], "| IS | ", labels2[output.argmax(dim=1).item()], " : ", labels2[test_label.item()])
            total_acc_test += acc
            i += 1

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [168]:
def evaluateWithoutAccuracy(model, test_data):
    test = TestDataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()
    
    resp = []
    output = None
    with torch.no_grad():
        i = 0
        for test_input in test_dataloader:
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            output = torch.round(nn.Softmax(dim=1)(output), decimals = 3)
            
            #print(tokenizer.decode(test_dataloader.dataset.texts[i].input_ids[0])[:100], " |||| ", labels2[output.argmax(dim=1).item()])
            resp.append(tokenizer.decode(test_dataloader.dataset.texts[i].input_ids[0])[:200] + " |||| " + labels2[output.argmax(dim=1).item()])
            i += 1
    return [resp, output]

In [ ]:
evaluate(model,df_test)

In [ ]:
evaluateAndPrintForModel(model, df_test)

In [158]:
text_to_add = []

In [173]:
while 1:
    query = input("Text to check:")
    query = query.replace(',', '').replace('\"', '').replace('\.', ' ').replace('\n', '').replace('\r', '').replace('\t','').replace('  ', '').strip().lower()
    if query == 'stop':
        break
    if len(query) < 25:
        continue
    df_test_input = pd.DataFrame({"text": [query]})
    evaluated_resp, tensor = evaluateWithoutAccuracy(model, df_test_input)
    evaluated_resp = evaluated_resp[0]
    output = evaluated_resp.split("|||| ")[1]
    print(output)
    metric = torch.topk(tensor, 2)
    firstOutput = metric[0][0][0].item()
    secondOutput = metric[0][0][1].item()
    overallScore = firstOutput + secondOutput
    howSure = (firstOutput / overallScore) * 100
    print(howSure)
    if howSure < 70:
        print("Sadly not sure...")
        continue
    text = input("Correct? [y/n]")
    if text == 'y':
        text_to_add.append(f'{output},{query}')
        print("Added, curr len: " + str(len(text_to_add)))
    else:
        new_output = input("New category:")
        text_to_add.append(f'{new_output},{query}')
        print("Added, curr len: " + str(len(text_to_add)))
        
        


Text to check: your information about midjourney is clear and concise. many thanks. recently i created amazing images with bluewillow and it's super cool. it's completely free. what do you think about bw


question
99.79979978610527


KeyboardInterrupt: Interrupted by user

In [19]:
for text in text_to_add:
    query = text.replace('\"', '').replace('\.', ' ').replace('\n', '').replace('\r', '').replace('\t','').replace('  ', '').strip().lower()
    print(query)

normal,i want to try runway. i think it is an awesome software!
question,does gpt 3 require a paid account? i tried to access it to no avail.
question,very nice content. thanks for the research. i would like to know if there is an ai application for predicting the next set of a group of numbers?or one that helps to quickly develop mobile apps?
question,this is so interesting!! it’s insane to see how technology develops. but would these ais like bhuman be able to read non-english names correctly or is that something to wait for?
question,quick question if anyone knows: how is the pricing determined on these illustrations? do you set your own or does adobe and the other ones which accept ai generated images decide?
extra,you also need a midjourney commercial license to sell the images generated there. otherwise they fall under the creative commons noncommercial 4.0 attribution international license (the “asset license”).
question,this is awesome thank you matt. question - do you know how